In [ ]:
#!git clone https://github.com/sobrad956/CMSC472FinalProject.git


In [ ]:
#!cd CMSC472FinalProject/


In [ ]:
import torch
import torchvision as tv
import torch.random
import os
import sys
from PIL import Image
from torchvision import transforms
from torchvision.datasets import Food101
from torchvision.transforms import ToTensor
import torchvision
import matplotlib.pyplot as plt
import matplotlib


In [ ]:
# assumes original color temp maxed out at (255,255,255)
# expect Tensor of [..., 3 (r,g,b), H, W]
class RandomSaltPepper(torch.nn.Module):
    def __init__(self, p=0.5, type='salt'):
        super().__init__()
        self.p = p
        self.salt = type in ['salt', 'both']
        self.pepper = type in ['pepper', 'both']
        self.color = []
        if self.salt:
            self.color.append(1)
        if self.pepper:
            self.color.append(0)

    def forward(self, imgs):
        if not len(self.color):
            return imgs

        # probably slow AF and not pythonic
        for i, img in enumerate(imgs):
            for h, row in enumerate(img[0]):
                for w, _ in enumerate(row):
                    if torch.rand(1) < self.p:
                        imgs[i, :, h, w] = self.color[torch.randint(
                            0, len(self.color), (1,)).item()]

        return imgs


class RandomGaussianNoise(torch.nn.Module):
    def __init__(self, mean=0, var=1):
        super().__init__()
        self.mean = mean
        self.var = var
        self.sigma = var ** 0.5

    def forward(self, imgs):
        noise = torch.normal(self.mean, self.sigma, imgs.shape)
        imgs += noise
        imgs = torch.clamp(imgs, 0, 1)
        return imgs


In [ ]:
# simple testbench
if __name__ == '__main__':
    root = os.path.expanduser(os.path.expanduser(os.path.join('~', 'data')))
    dataset = Food101(root=root, download=True, transform=ToTensor())
    aug = RandomSaltPepper(p=0.2, type='both')
    aug2 = RandomGaussianNoise(mean=0, var=1./255.)
    oim = torch.unsqueeze(dataset[0][0], 0)
    aim = aug(torch.clone(oim))
    aim2 = aug2(torch.clone(oim))
    plt.figure(0)
    plt.imshow(aim[0].permute(1, 2, 0))
    plt.figure(1)
    plt.imshow(aim2[0].permute(1, 2, 0))
    plt.figure(2)
    plt.imshow(oim[0].permute(1, 2, 0))
    plt.show()


In [ ]:
len(dataset)


In [ ]:
# Set up dataloaders
import torchvision.transforms as transforms
from torch.utils.data import DataLoader


normMean = [0.49139968, 0.48215827, 0.44653124]
normStd = [0.24703233, 0.24348505, 0.26158768]
normTransform = transforms.Normalize(normMean, normStd)

trainTransform = transforms.Compose([
    transforms.Resize([32, 32]),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normTransform
])

testTransform = transforms.Compose([
    transforms.Resize([32, 32]),
    transforms.ToTensor(),
    normTransform
])

trainLoader = DataLoader(Food101(root=root, split="train", download=False,
                         transform=trainTransform), batch_size=64, shuffle=True)
valLoader = DataLoader(Food101(root=root, split="test", download=False,
                       transform=testTransform), batch_size=64, shuffle=False)


In [ ]:
# Set up all trial augmentations
from augmentations import expt1augs, expt2augs, expt3augs, expt4augs, expt5augs, expt6augs

# Baseline
baseline = [(trainTransform, 'baseline')]

# Experiment 1: Random S&P Noise
noisepb = [.10, .20, .40]
expt1 = [
    (
        expt1augs.RandomSaltPepper(p, type='both'),
        f'expt1-rsp-{int(p * 100)}'
    )
    for p in noisepb
]

# Experiment 2: Temperature Changes
temps = [1000, 3000, 6000, 9000]  # kelvin color temp
ps = [0.5]  # 50% chance
expt2 = [
    (
        expt2augs.RandomAdjustTemp(color=t, p=p),
        f'expt2-temp-{t}-p{int(p * 100)}'
    )
    for t in temps for p in ps
]

# Experiment 3: Random Dropout of Regions
ks = [0.5]  # prob of augment
nps = [1, 2, 5]  # num patches
ss = [5]  # size of patches
expt3 = [
    (
        expt3augs.RandomDropout(s, n, k),
        f'expt3-dropout-p{int(k * 100)}-i{n}-s{s}'
    )
    for n in nps for k in ks for s in ss
]

# Experiment 4: Random Region Swap
ks = [0.5]  # prob of augment
nps = [2, 4, 6]  # num patches
ss = [5]  # size of patches
expt4 = [
    (
        expt4augs.RandomRegionSwap(s, n, k),
        f'expt4-swap-p{int(k * 100)}-i{n}-s{s}'
    )
    for n in nps for k in ks for s in ss
]

# Experiment 5: Black Bar Occlusion
ns = [5]  # num of contiguous rows
ms = [5]  # num of contiguous columns
ks = [0.5]  # prob of augment
expt5 = [
    (
        expt5augs.RandomRowColDropout(n, m, k),
        f'expt5-rowcol-p{int(k * 100)}-n{n}-m{m}'
    )
    for n in ns for m in ms for k in ks
]

# Experiment 6: Random Apply Rotation, Scaling, Shearing, Translation
augs = [
    transforms.RandomRotation(180),
    transforms.RandomResizedCrop((32, 32), antialias=True),
    transforms.RandomAffine(0, translate=(0.20, 0.20)),
    transforms.RandomAffine(0, shear=10)
]  # one augment is applied in this list
ks = [0.5]  # prob of exactly one above aug being applied
expt6 = [
    (
        expt6augs.RandomApplyOne(k=k, augs=augs),
        f'expt6-applyone-p{int(k * 100)}'
    )
    for k in ks
]


In [ ]:
# Run Densenet/SqueezeNet/InceptionNet3
# import models

from models import RunNet

# change this to whatever set of experiments that will be done
exptTransform = expt1
model = 'DenseNet' 
# model = 'SqueezeNet' 
# model = 'InceptionNet'

for expt_transform, expt_name in exptTransform:
    r = RunNet(model=model, trainLoader=trainLoader,
               valLoader=valLoader, verbose=True, experiment=expt_name, nEpochs=2, augment=expt_transform)
    r.run()
